# Scraping data from subreddit <a href='https://www.reddit.com/r/AskDocs/'>r/AskDocs</a> 🩺🥼

In [102]:
import praw
import pandas as pd
from datasets import Dataset
import yake
import math

In [103]:
reddit=praw.Reddit(user_agent=True,
                   client_secret='zS6cuhXCP0b1HCU0aSXl236vu9o9SA',client_id='OzzaZ-aY_0oo4TnMg0fnWA',username='EastLeft',password='randomaccnt278')

In [104]:
def genuineQues(title):
    title=title.lower()
    if('help' in title or 'medical' in title or 'symptoms' in title):
        return 1
    return -1
def genuineComment(reply,role,author):
    
    if(reply=='[deleted]' or reply =='[removed]' or (author=='AutoModerator')):
        return -1
    return 1
def getkeyword(text):
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size =1
    deduplication_threshold = 0.9
    numOfKeywords = 1
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    return keywords[0]

In [106]:
%%time
subreddit = reddit.subreddit('askdocs')
# Define lists to store data
data = []

# Scraping posts & Comments
for post in subreddit.top(limit= 50000): 
    if(genuineQues(post.title)==1):
        data.append({
            'Type': 'Post',
            'Post_id': post.id,
            'Title': post.title,
            'Author': post.author.name if post.author else 'Unknown',
            'Role':post.author_flair_text,
            'Timestamp': post.created_utc,
            'Text': post.selftext,
            'Score': post.score,
            'Total_comments': post.num_comments,
            'Post_URL': post.url
        })

    # Check if the post has comments
        if post.num_comments > 0:
            c=reddit.submission(url=post.url)
            c.comment_limit=3
            c.comments.replace_more(limit= 0)
            for comment in c.comments:
                data.append({
                'Type': 'Comment',
                'Post_id': post.id,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Role':comment.author_flair_text,
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Total_comments': 0,
                'Post_URL': None
                })

for post in subreddit.hot(limit= 50000): 
    if(genuineQues(post.title)==1):
        data.append({
            'Type': 'Post',
            'Post_id': post.id,
            'Title': post.title,
            'Author': post.author.name if post.author else 'Unknown',
            'Role':post.author_flair_text,
            'Timestamp': post.created_utc,
            'Text': post.selftext,
            'Score': post.score,
            'Total_comments': post.num_comments,
            'Post_URL': post.url
        })

    # Check if the post has comments
        if post.num_comments > 0:
            c=reddit.submission(url=post.url)
            c.comment_limit=3
            c.comments.replace_more(limit= 0)
            for comment in c.comments:
                data.append({
                'Type': 'Comment',
                'Post_id': post.id,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Role':comment.author_flair_text,
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Total_comments': 0,
                'Post_URL': None
                })


# Create pandas DataFrame for posts and comments
df = pd.DataFrame(data)
df.head(10)

CPU times: user 440 ms, sys: 53 ms, total: 493 ms
Wall time: 2min 8s


,Type,Post_id,Title,Author,Role,Timestamp,Text,Score,Total_comments,Post_URL
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,1594030661.0,"Dear users,\n\nRecently there have been nursin...",2779,1,https://www.reddit.com/r/AskDocs/comments/hm52...
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,1681872922.0,I’m the person who posted yesterday because my...,2759,23,https://www.reddit.com/r/AskDocs/comments/12re...
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,None,2023-04-19 02:55:23,Thank you for your submission. **Please note t...,1,0,None
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,2023-04-19 03:34:33,Thank you for the update!! Best wishes to both...,596,0,None
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,None,2023-04-19 02:56:41,[deleted],678,0,None
5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,None,1596052554.0,Original Post:\n\n [https://www.reddit.com/r/A...,2493,11,https://www.reddit.com/r/AskDocs/comments/i07g...
6,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,Eastern_Concentrate8,Layperson/not verified as healthcare professio...,2020-07-29 20:06:30,So glad you found an answer and are on the way...,248,0,None
7,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,jcarberry,Physician | Moderator,2020-07-29 21:26:42,"Thank you for updating, OP! Best of luck in yo...",1,0,None
8,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,graye1999,This user has not yet been verified.,2020-07-29 20:15:35,That’s fascinating. I would have had a similar...,107,0,None
9,Post,e3nrbd,Need help - Harvard gave up - homeless,gooddiagnostician,Layperson/not verified as healthcare professio...,1575073307.0,Please take the time to read this. I am extre...,2128,236,https://www.reddit.com/r/AskDocs/comments/e3nr...


In [107]:
df.to_csv("askdocs.csv")

# Data preprocessing

In [108]:
import praw
import pandas as pd
from datasets import Dataset

In [109]:
df=pd.read_csv('askdocs.csv')
df

,Unnamed: 0,Type,Post_id,Title,Author,Role,Timestamp,Text,Score,Total_comments,Post_URL
0,0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,1594030661.0,"Dear users,\n\nRecently there have been nursin...",2779,1,https://www.reddit.com/r/AskDocs/comments/hm52...
1,1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,1681872922.0,I’m the person who posted yesterday because my...,2759,23,https://www.reddit.com/r/AskDocs/comments/12re...
2,2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,2023-04-19 02:55:23,Thank you for your submission. **Please note t...,1,0,NaN
3,3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,2023-04-19 03:34:33,Thank you for the update!! Best wishes to both...,596,0,NaN
4,4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,2023-04-19 02:56:41,[deleted],678,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
648,648,Comment,1ce2hy1,Can someone help me understand my CT?,AutoModerator,NaN,2024-04-27 01:32:45,Thank you for your submission. **Please note t...,1,0,NaN
649,649,Post,1ce27p0,Could anyone help me identify this?,feelsprofound,Layperson/not verified as healthcare professio...,1714180698.0,Recently noticed this on my 3 year old brother...,1,1,https://www.reddit.com/r/AskDocs/comments/1ce2...
650,650,Comment,1ce27p0,Could anyone help me identify this?,AutoModerator,NaN,2024-04-27 01:18:18,Thank you for your submission. **Please note t...,1,0,NaN
651,651,Post,1ce1a3x,Gastro docs please help!,CauliflowerWeak8996,Layperson/not verified as healthcare professional,1714177941.0,I’m 23yo female no extensive medical history n...,0,1,https://www.reddit.com/r/AskDocs/comments/1ce1...


In [110]:
df = df.drop('Timestamp',axis=1)
df = df.drop('Total_comments', axis=1)
df = df.drop('Post_URL', axis=1)
df

,Unnamed: 0,Type,Post_id,Title,Author,Role,Text,Score
0,0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users,\n\nRecently there have been nursin...",2779
1,1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,I’m the person who posted yesterday because my...,2759
2,2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,Thank you for your submission. **Please note t...,1
3,3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank you for the update!! Best wishes to both...,596
4,4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,[deleted],678
...,...,...,...,...,...,...,...,...
648,648,Comment,1ce2hy1,Can someone help me understand my CT?,AutoModerator,NaN,Thank you for your submission. **Please note t...,1
649,649,Post,1ce27p0,Could anyone help me identify this?,feelsprofound,Layperson/not verified as healthcare professio...,Recently noticed this on my 3 year old brother...,1
650,650,Comment,1ce27p0,Could anyone help me identify this?,AutoModerator,NaN,Thank you for your submission. **Please note t...,1
651,651,Post,1ce1a3x,Gastro docs please help!,CauliflowerWeak8996,Layperson/not verified as healthcare professional,I’m 23yo female no extensive medical history n...,0


In [111]:
df.drop(df[df.Author =='AutoModerator'].index,inplace=True)
df.drop(df[df.Text =='[deleted]'].index,inplace=True)
df.reset_index()

,index,Unnamed: 0,Type,Post_id,Title,Author,Role,Text,Score
0,0,0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users,\n\nRecently there have been nursin...",2779
1,1,1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,I’m the person who posted yesterday because my...,2759
2,3,3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank you for the update!! Best wishes to both...,596
3,5,5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post:\n\n [https://www.reddit.com/r/A...,2493
4,6,6,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,Eastern_Concentrate8,Layperson/not verified as healthcare professio...,So glad you found an answer and are on the way...,248
...,...,...,...,...,...,...,...,...,...
484,645,645,Comment,1cdxxpf,Please help me figure out what this is!,ssin14,Registered Nurse,What is your question? The location of the ble...,2
485,646,646,Comment,1cdxxpf,Please help me figure out what this is!,Firstsassythensweet,Layperson/not verified as healthcare professional,https://preview.redd.it/miwe9ec8cwwc1.jpeg?wid...,1
486,647,647,Post,1ce2hy1,Can someone help me understand my CT?,Odd-Application-6597,Layperson/not verified as healthcare professional,"Female, 29yo just had a head and neck ct, \nWh...",0
487,649,649,Post,1ce27p0,Could anyone help me identify this?,feelsprofound,Layperson/not verified as healthcare professio...,Recently noticed this on my 3 year old brother...,1


In [112]:

df['Role'] = df['Role'].where(df['Role'] != 'Layperson/not verified as healthcare professional', 'Patient')
df['Role'] = df['Role'].where(df['Role'] != 'This user has not yet been verified.', 'Patient')
df

,Unnamed: 0,Type,Post_id,Title,Author,Role,Text,Score
0,0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users,\n\nRecently there have been nursin...",2779
1,1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,I’m the person who posted yesterday because my...,2759
3,3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank you for the update!! Best wishes to both...,596
5,5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post:\n\n [https://www.reddit.com/r/A...,2493
6,6,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,Eastern_Concentrate8,Layperson/not verified as healthcare professio...,So glad you found an answer and are on the way...,248
...,...,...,...,...,...,...,...,...
645,645,Comment,1cdxxpf,Please help me figure out what this is!,ssin14,Registered Nurse,What is your question? The location of the ble...,2
646,646,Comment,1cdxxpf,Please help me figure out what this is!,Firstsassythensweet,Patient,https://preview.redd.it/miwe9ec8cwwc1.jpeg?wid...,1
647,647,Post,1ce2hy1,Can someone help me understand my CT?,Odd-Application-6597,Patient,"Female, 29yo just had a head and neck ct, \nWh...",0
649,649,Post,1ce27p0,Could anyone help me identify this?,feelsprofound,Layperson/not verified as healthcare professio...,Recently noticed this on my 3 year old brother...,1


In [99]:
i=0
while(i<len(df)-1):
    if(df['Type'][i]=='Post'):
        j=i+1
        while(j<len(df) and df['Type'][j]=='Comment'):
            reply=df['Text'][j]
            role=df['Role'][j]
            author=df['Author'][j]
            if((genuineComment(reply,role,author))==-1):
                df.drop(j)
            j+=1
    i+=1
df

,Unnamed: 0,Type,Post_id,Title,Author,Role,Text,Score
0,0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users,\n\nRecently there have been nursin...",2776
1,1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,I’m the person who posted yesterday because my...,2760
2,2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,Thank you for your submission. **Please note t...,1
3,3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank you for the update!! Best wishes to both...,598
4,4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,[deleted],677
5,5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post:\n\n [https://www.reddit.com/r/A...,2497
6,6,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,Eastern_Concentrate8,Layperson/not verified as healthcare professio...,So glad you found an answer and are on the way...,249
7,7,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,jcarberry,Physician | Moderator,"Thank you for updating, OP! Best of luck in yo...",1
8,8,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,graye1999,This user has not yet been verified.,That’s fascinating. I would have had a similar...,104
9,9,Post,1ceap60,"Itchy anus, causing me to insert my finger in ...",throwitaway1231082,Layperson/not verified as healthcare professional,"27M 190lbs 183cm. For sometime now, and I me...",323


One challenge is that the ideal answer could be quite inside a conversation of a thread.

# Fine Tuning